In [1]:
# Libraries
import pandas as pd
import random

#### Functions

In [2]:
def menor_distancia(nao_visitados, distancias, origem):
    """
    """

    max_distance = max(distancias.values()) * 2

    for nv in nao_visitados:
        if distancias[(origem, nv)] < max_distance:
            max_distance = distancias[(origem, nv)]
            proximo = nv

    return proximo   

#### Parameters

<!-- 

---------------------------------------------------------------------------------------------------------------------------

df['Customers'] = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11',]
df['Log'] = [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2]
df['Lat'] = [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3,]

---------------------------------------------------------------------------------------------------------------------------
  
df['Customers'] = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5']
df['Log'] = [0, 1, 1, -1, -1, 0]
df['Lat'] = [0, 1, 2, 2, 1, 3]


-->

In [10]:
# Number of Customers 
total_customers = 50

# Number of Sellers / Trucks
total_sellers = 1

# Seller capacity
sellers_capacity = 28800 # s (8h/day)

# Truck capacity
truck_capacity = 60 # unit (High Pressure Cylinders)

# Truck Speed
velocity = 13.9 # m/s (Normal speed of a Truck (50 km/h))

# Dutarion Service
change_duration = 300 # 5 minutes on average to change a cylinder
service_duration = 600 # 5 minutes per stop

# ID Source
warehouse = 0
 
# the bigM
bigM = total_customers + 1

# Bidirectional Graph: G(V,A)
# (C[i], C[j]) : Cost

df = pd.DataFrame()

# C0 = Warehouse
customers = ['C0']
long = [0]
lat = [0]
cyn = [0]
val = [0]

X, Y  = 2600, 2600
# Generate values randomly to create the scenario
for c in range(1, total_customers+1):
    customers.append(f'C{c}')
    long.append(0 + X * random.random())
    lat.append(0 + Y * random.random())
    cyn.append( random.choice(range(1, 10)) )
    val.append( random.choice(range(80, 1500)) * cyn[-1])

df['Customers'] = customers
df['Log'] = long
df['Lat'] = lat  
df['Cyn'] = cyn
df['Val'] = val


# Calculate the euclidian distance between each point on the map
cost = {}
for i in df.index:
    for j in df.index:

        x = df.loc[i, 'Log'] - df.loc[j, 'Log']
        y = df.loc[i, 'Lat'] - df.loc[j, 'Lat']
        cost[(i, j)] = (x**2 + y**2)**(0.5)



#### Gready Search

In [19]:
nao_visitados = list(range(1, total_customers+1))
resultados = {}

for ts in range(total_sellers):
    
    truck, seller = 0, 0
    rota, profit = [0], 0
    temp = True
        
    while temp:

        if (len(nao_visitados) != 0):
            
            next_node = menor_distancia(nao_visitados, cost, rota[-1])
            truck_aux = truck + df.loc[next_node, 'Cyn']
            seller_aux = seller + (cost[(rota[-1], next_node)] / velocity) + (change_duration * df.loc[next_node, 'Cyn']) + service_duration + (cost[(next_node, 0)] / velocity)

            if (seller_aux <= sellers_capacity) & (truck_aux <= truck_capacity):

                nao_visitados.remove(next_node)
                rota.append(next_node)
                seller += (cost[(rota[-1], next_node)] / velocity) + change_duration + service_duration
                truck += df.loc[next_node, 'Cyn']
                profit += df.loc[next_node, 'Val']

            else:
                seller += (cost[(rota[-1], 0)] / velocity)
                temp = False
        else:
            temp = False
            
    resultados[ts] = {
                        'Rotas' : rota,
                        'Lucro' : profit,
                        'Carga' : truck,
                        'Tempo' : seller,
    }